In [ ]:
# standard libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext autoreload
%autoreload 2

# own functions

import proj1_helpers as P1H
import dataprocessing as DP
import methods as ME
import cross_validation as CV
from grad_loss import*

#constants
train_path = 'train.csv'
test_path = 'test.csv'

In [ ]:
orig_y, orig_x, orig_ids = load_csv_data(train_path, sub_sample=False) #remember to switch of subsample when running it "for real"
pred_y, pred_x, pred_ids = load_csv_data(test_path, sub_sample=False)

In [ ]:
# To provide clarity to which data I am processing
x = np.copy(orig_x)

# Cleans values that are -999 to zero, mean and median
no_clean = np.copy(orig_x)
clean_zero = DP.clean_data(x)
clean_mean = DP.clean_data(x, replace_no_measure_with_mean=True)
clean_medi = DP.clean_data(x, replace_no_measure_with_median=True)

# Make array to test for later
cleanDataArray = [no_clean, clean_zero, clean_mean, clean_medi]

In [ ]:
## Normalizing data:
normalizedDataArray=[]
for i, data in enumerate(cleanDataArray):
    normalizedDataArray.append(DP.normalize(data))
  

In [ ]:
lambda_=2.33572146909e-05 #taken from exploration of basic methods
k_folds=5
avg_losses=[]
avg_preds_all=[]
for data in normalizedDataArray:
    avg_loss, losses, avg_preds, pred_acc_percents = CV.cross_validation(ME.ridge_regression, orig_y, data, k_folds, lambda_)
    avg_losses.append(avg_loss)
    avg_preds_all.append(avg_preds)
print("this is lambda", lambda_)
print("this is average losses: ",avg_losses)
print("this is average prediction error:",avg_preds_all)

In [ ]:
##Based on this, we choose to continue with the data where all missing values are replaced by 0. 

chosenData=normalizedDataArray[2]
print(chosenData.shape)

In [ ]:
## Now we want to performe PCA on the chosen data
numberOfDimensions=(30,29,28,27,26,25,24,23,22,21,20)
pcas=[]
for i, degree in enumerate(numberOfDimensions):
    pca_i=DP.pca(chosenData,degree)[0]
    pcas.append(pca_i)

In [ ]:
lambda_=2.33572146909e-05 #taken from exploration of basic methods
k_folds=5
avg_losses=[]
avg_preds_all=[]
for data in pcas:
    avg_loss, losses, avg_preds, pred_acc_percents = CV.cross_validation(ME.ridge_regression, orig_y, data, k_folds, lambda_)
    avg_losses.append(avg_loss)
    avg_preds_all.append(avg_preds)
print("this is lambda", lambda_)
print("this is average losses: ",avg_losses)
print("this is average prediction error:",avg_preds_all)

In [ ]:
minimum_loss=np.min(avg_losses)
print(minimum_loss)

chosenData=pcas[1]

In [ ]:
#Now we want to pick a polynomial degree:

In [ ]:
degrees=(3,4,5,6,7,8,9,10,11,13,15,17,19,20)
lambdas=np.logspace(-9,1,15)
min_loss=1000;
min_degree=0;
min_lambda=0
max_acc=0
avg_losses=np.zeros((len(degrees),len(lambdas)))
avg_acc=np.zeros((len(degrees),len(lambdas)))
for d,degree in enumerate(degrees):
    phi=DP.build_poly(chosenData,degree)
    for l,lambda_ in enumerate(lambdas):
        avg_loss, losses, avg_preds, pred_acc_percents = CV.cross_validation(ME.ridge_regression, orig_y, phi, k_folds, lambda_)
        avg_losses[d,l]=avg_loss
        avg_acc[d,l]=avg_preds
        if avg_loss < min_loss:
            min_loss=avg_loss
            min_degree=degree
            min_lambda=lambda_
            max_acc=avg_preds
            

In [ ]:
print(min_loss)
print(min_degree)
print(min_lambda)
print(max_acc)

In [ ]:
#based on this, I want to choose a polynomial of degree 
#0.781696843395 
#7
#3.72759372031e-06
#0.20651999999999998

chosenData=DP.build_poly(chosenData,min_degree)
lambda_=min_lambda

In [ ]:
clean_test_x=DP.clean_data(pred_x)
clean_train_x=clean_data(orig_x)

norm_test_x=DP.normalize(clean_test_x)
norm_train_x=DP.normalize(clean_train_x)

pca_test=DP.pca(norm_test_x,30)[0]
pca_train=DP.pca(norm_train_x,30)[0]

phi_test=DP.build_poly(pca_test,7)
phi_train=DP.build_poly(pca_train,7)

lambda_=3.72759372031e-06
degree=7

loss,w=ME.ridge_regression(orig_y,phi_train,lambda_)


In [ ]:
print(w.shape)

In [ ]:
y_predicted=P1H.predict_labels(w,phi_test)

In [ ]:
print(y_predicted.shape)

In [ ]:
print(y_predicted)

In [ ]:
name="testing_hedda.csv"

create_csv_submission(pred_ids, y_predicted, name)